In [1]:
import pandas as pd
import numpy as np
from numpy.random import *
import datetime
import math
import os
import matplotlib.pyplot as plt
from pylab import rcParams

plt.style.use('ggplot')
%pylab inline --no-import-all

def row(x):    #行
    return x.reshape(1,-1)

def column(x):     #列
    return x.reshape(-1,1)

def row2array(x):
    return x[0,:]

def column2array(x):
    x=x.T
    return x[0,:]

def lorenz(v, t):   #列ベクトルをarrayに変換してからぶち込む
    x=v[0]
    y=v[1]
    z=v[2]
    s = 10
    r = 28
    b = 8/3
    return np.array([-s*x-y, r*x-y-x*z, -b*z+x*y])
 
def myoutput(x,t):   #xには、列ベクトルを入れること
    rand = np.random.normal(0, 1, (3, 1))
    dt=0.0001
    x=x + 0.1*rand + dt*dt*rand + dt*rand
    return column(x)


Populating the interactive namespace from numpy and matplotlib


##  Here, we perform estimation simulation using nonlinear dynamics called Lorenz equation.

$$
    \sigma = 10, r = 28, b = \frac{8}{3}
$$

$$
    {\dot{x} = \sigma(y-x)\\
\dot{y} = rx -y-z\\
\dot{z} = -bz + xy
}
$$

[  0.00000000e+00   1.00000000e-01   2.00000000e-01 ...,   2.99800000e+02
   2.99900000e+02   3.00000000e+02]
(3001,)
(3, 3001)


In [2]:
#分散の初期設定
Rtr = 0.3
R = np.diag([0.5, 0.2, 0.1]) #3D
Q = np.diag([2, 4, 3])  #3D
dt = 0.1

# 真値の計算
t = np.linspace(0,300,300/dt+1,dtype=float)#時間　dt=0.01の間隔で0から10秒までの配列
xtr0 = column(np.array([0.1, 0.5, 1.2]))#2D列 #真値の初期値、行列xtr0の1番目の行
ytr0 = myoutput(xtr0,t[0])#2D_列#真値の出力
w0 = np.dot(Q ,np.random.normal(0, 1, (3, 1)))

xtr = xtr0
ytr = ytr0 
w = w0
yt0 = myoutput(xtr0,t[0])#2D_列
yt = yt0

for i in range(1,len(t)):
    k1 = dt * lorenz(xtr[:,i-1], t[i-1] ) #array
    k2 = dt * lorenz(xtr[:,i-1] + k1/2, t[i-1]+dt/2 )   #array
    k3 = dt * lorenz(xtr[:,i-1] + k2/2, t[i-1]+dt/2)   #array
    k4 = dt * lorenz(xtr[:,i-1] + k3,t[i-1]+dt)    #array
    
    w_i = np.dot(Q ,np.random.normal(0, 1, (3, 1)))#3x3と行ベクトルの内積で列ベクトルになる　
    w = np.concatenate([w,w_i],axis=1)    
    
    xtr_i = column(xtr[:,i-1]  +  (k1+2*k2+2*k3+k4) /6 ) +column(w[:,i-1].T*dt)
    xtr = np.concatenate([xtr,xtr_i],axis=1) #列を横に並べる

    ytr_i = myoutput(column(xtr[:,i-1]),t[i-1])#2D_列
    ytr = np.concatenate([ytr,ytr_i],axis=1) #列を横に並べる
    
    ###   観測値の計算   ###
    yt_i = ytr_i+column(np.dot(R,np.random.normal(0, 1, (3, 1))))  
    yt = np.concatenate([yt,yt_i],axis=1)    #3D_行
    

len_y = ytr.shape[0]
len_x = ytr.shape[1]


print(t)
print(t.shape)
print(yt.shape)

# 配列  をプロット
plt.plot(t)
plt.show()

In [3]:
#  EnKFの初期設定
M = 100   #サンプルの数
barx0 = column(np.array([2,10,1]))  #状態の初期値 2D_列
P0 = np.diag([1, 3, 3])  #共分散行列の初期値  3D

# アンサンブル行列X_0/tの作成
x0=column(barx0) #2D_列
# アンサンブル行列Y_0/tの作成
y0 = myoutput( column(x0[:,0]), t[0])  
for i in range(M-1):
        x0_i=barx0 + np.dot(Q,np.random.normal(0, 1, (3, 1))) #2D_列　正規乱数
        x0 = np.concatenate([x0,x0_i],axis=1) #3D_列
        y0_i=myoutput( column(x0[:,0]), t[0]) + np.dot(Q,np.random.normal(0, 1, (3, 1))) 
        y0 = np.concatenate([y0,y0_i],axis=1) #3D_列

 


print(x0.shape)
print(y0.shape)

(3, 100)
(3, 100)


In [4]:
#Ensemble Kalman Filterの計算

#誤差アンサンブル行列の作成

yest=column(ytr[:,0])

for i in range(len(t)-1):
    x=np.array([[0],[0],[0]])
    ### EnKF: 観測更新ステップ ###
    xhat = column(numpy.sum(x0,axis=1) / M ) #2D_列
    yhat = column(numpy.sum(y0,axis=1) / M ) #2D_列
    yest_i = yhat
    yest = np.concatenate([yest,yest_i],axis=1) #3D_列
    Pxy = np.dot( (np.kron(np.eye(1,100),xhat) - x0), (np.kron(np.eye(1,100),yhat) - y0).T ) / (M-1)  #(3x3)  U 共分散行列
    Pyy = np.dot( (np.kron(np.eye(1,100),yhat) - y0), (np.kron(np.eye(1,100),yhat) - y0).T ) / (M-1)  #(3x3)  V 共分散行列
    K = np.dot(Pxy, np.linalg.inv(Pyy))     #np.linalg.inv(A) Aの逆行列 カルマンゲイン  #(3x3)
    
    ### EnKF: 予測更新ステップ  ###
    for j in range(M-1):
        x_j = column( x0[:,j] + np.dot(K, yt[:,i] - y0[:,j]) ) #2D_列
        x = np.concatenate([x,x_j],axis=1) #3D_列
        x0[:,j] = x[:,j] + column2array(row(lorenz(x[:,j], t[i])).T*dt +np.dot(Q,np.random.normal(0, 1, (3, 1))) * dt )  #array型
        
        y0[:,j] = column2array(  myoutput(column(x0[:,j]),t[i]) + np.dot(R, np.random.normal(0, 1, (3, 1)) ) )    #array型

            

In [5]:
print(t.shape)  #(201,) tの真値
print(xtr.shape)  #(3, 201) xの真値
print(ytr.shape)  #(3, 201) yの真値
print(yest.shape)  #(3, 201) yestの真値 #便宜上t=0　を挿入している。

(3001,)
(3, 3001)
(3, 3001)
(3, 3001)


In [6]:
df = pd.DataFrame({ 't' : t ,
                    'xtr1' : xtr[0,:],
                    'xtr2' : xtr[1,:],
                    'xtr3' : xtr[2,:],
                    'ytr1' : ytr[0,:],
                    'ytr2' : ytr[1,:],
                    'ytr3' : ytr[2,:],
                    'yest1' : yest[0,:],
                    'yest2' : yest[1,:],
                    'yest3' : yest[2,:],
                  })

df = df.set_index(['t'])

In [7]:
df1 = pd.DataFrame({ 't' : t ,
                    'yest_x' : yest[0,:],
                    'yest_y' : yest[1,:],
                    'yest_z' : yest[2,:],
                  })
df1 = df1.set_index(['t'])

In [15]:
df1.plot(subplots=True, figsize=(24, 24),ylim=[-3, 3]); plt.legend(loc='best')
fig = plt.figure()

plt.show()
